In [1]:
# Import Necessary Modules

import glob
import matplotlib.pyplot as plt 
from skimage import io
import numpy as np
%matplotlib inline 

In [2]:
#This function reads in all n images in catsfolder/ and dogsfolder/. 
#Each 64 x 64 image is reshaped into a length-4096 row vector. 
#These row vectors are stacked on top of one another to get a data matrix
#X of size n x 4096. We also generate a -1 label if the row vector corresponds
#to a cat image and a +1 label if the row vector corresponds to a dog image
#and stack these on top of one another to get a label vector y of length n.

def read_data():
    
    #get image filenames
    cat_locs = glob.glob('catsfolder/*.jpg')
    dog_locs = glob.glob('dogsfolder/*.jpg')
    num_cats = len(cat_locs)
    num_dogs = len(dog_locs)

    #initialize empty arrays
    X_cats = np.zeros((num_cats,64*64))
    X_dogs = np.zeros((num_dogs,64*64))
    y_cats = np.zeros((num_cats,1))
    y_dogs = np.zeros((num_dogs,1))
               
    #Load data, reshape into a 1D vector and set labels
    
    keep_track = 0

    for i in range(len(cat_locs)):
        img = cat_locs[i]
        im = io.imread(img)
        im = im.reshape(64*64)
        X_cats[i,:] = im
        y_cats[i] = -1.0
        keep_track += 1

    for i in range(len(dog_locs)):
        img = dog_locs[i]
        im = io.imread(img)
        im = im.reshape(64*64)
        X_dogs[i,:] = im
        y_dogs[i] = 1.0
        keep_track += 1
    
    # combine both datasets
    X = np.append(X_cats,X_dogs,0)
    y = np.append(y_cats,y_dogs)
    
    return X, y 

In [4]:
#This function takes in a vector of true labels ytrue
#and a vector of guessed labels yguess and reports back
#the accuracy of the guesses out of 100%.

def calculate_accuracy(ytrue, yguess):
    
    # compare your predictions with the correct labels to determine how many of your predictions were correct.
    correct = sum((ytrue == yguess))
    total = len(ytrue)
    accuracy = 100*float(correct)/float(total)
    # divide the number of correct predictions by the number of total samples to determine your classification accuracy.
    return accuracy

In [5]:
#This function takes in a data matrix X, label vector y, and 
#the desired percentage testpercent. It randomly assigns  
#testpercent of the rows of X and y to Xtest and ytest. 
#The remaining data is assigned to Xtrain and ytrain.

def split_data(X,y,testpercent):
        
    [n, d] = X.shape
    
    ntest = int(round(n*(float(testpercent)/100)))
    ntrain = int(round(n - ntest))
        
    Xtrain = np.zeros((ntrain,d))
    Xtest = np.zeros((ntest,d))
    ytrain = np.zeros((ntrain,1))
    ytest = np.zeros((ntest,1))   
        
    Data = np.column_stack((X,y))
    Data = np.random.permutation(Data)
    
    for i in range(ntest):
        Xtest[i,:] = Data[i,0:d]
        ytest[i] = Data[i,d]
        
    for i in range(ntrain):
        Xtrain[i,:] = Data[i+ntest,0:d]
        ytrain[i] = Data[i+ntest,d]
        
    return Xtrain, ytrain, Xtest, ytest

In [17]:
def cnn(Xtrain,ytrain,Xrun):
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
    from tensorflow.keras.layers import Conv2D, MaxPooling2D
    import os
    os.environ['KMP_DUPLICATE_LIB_OK']='True'
    tf.keras.backend.clear_session()
    p = np.random.permutation(len(Xtrain))
    X_=Xtrain[p]
    y_=ytrain[p]
    X_=np.array(X_).reshape(-1,64,64,1)
    X_ = X_/255.0
    y_=(y_+1)/2
    model = Sequential()

    model.add(Conv2D(64, (3, 3), input_shape=X_.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten()) 

    model.add(Dense(64))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    model.fit(X_, y_, batch_size=32, epochs=10, validation_split=0.1)
    result=model.predict(np.array(Xrun).reshape(-1,64,64,1)/255.0)
    yguess=((result>0.5).astype(float)-0.5)*2
    result_train=model.predict(np.array(Xtrain).reshape(-1,64,64,1)/255.0)
    yguess_train=((result_train>0.5).astype(float)-0.5)*2
    return yguess,yguess_train

In [18]:
#main function
X,y = read_data()
Xtrain, ytrain, Xtest, ytest=split_data(X,y,20)
yguess,yguess_train=cnn(Xtrain,ytrain,Xtest)
print("training accuracy",calculate_accuracy(ytrain, yguess_train))
print("test accuracy",calculate_accuracy(ytest, yguess))

Train on 1440 samples, validate on 160 samples
Epoch 1/10
1440/1440 [==============================] - 7s 5ms/sample - loss: 0.5816 - accuracy: 0.7576 - val_loss: 0.2997 - val_accuracy: 0.8938
Epoch 2/10
1440/1440 [==============================] - 6s 4ms/sample - loss: 0.1947 - accuracy: 0.9312 - val_loss: 0.1303 - val_accuracy: 0.9625
Epoch 3/10
1440/1440 [==============================] - 6s 4ms/sample - loss: 0.1135 - accuracy: 0.9618 - val_loss: 0.0966 - val_accuracy: 0.9625
Epoch 4/10
1440/1440 [==============================] - 6s 4ms/sample - loss: 0.0811 - accuracy: 0.9729 - val_loss: 0.0514 - val_accuracy: 0.9875
Epoch 5/10
1440/1440 [==============================] - 6s 4ms/sample - loss: 0.0516 - accuracy: 0.9826 - val_loss: 0.0519 - val_accuracy: 0.9875
Epoch 6/10
1440/1440 [==============================] - 7s 5ms/sample - loss: 0.0523 - accuracy: 0.9833 - val_loss: 0.0584 - val_accuracy: 0.9875
Epoch 7/10
1440/1440 [==============================] - 6s 4ms/sample - loss: